<a href="https://colab.research.google.com/github/carloseducorinto/LangChain-for-LLM-Application-Development/blob/main/L3_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

import warnings
warnings.filterwarnings('ignore')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [8]:
llm_model = "gpt-3.5-turbo"

In [ ]:
#!pip install pandas
#!pip install langchain langchain_community
#!pip install -U langchain-openai

In [4]:
import pandas as pd
df = pd.read_csv('data.csv')

In [5]:
df.head()

,Product,Review
0,Smartphone X100,Excellent battery life and fast performance.
1,Laptop Pro 15,Great display but battery life could be better.
2,Wireless Headphones,Sound quality is top-notch and very comfortable.
3,4K TV UltraHD,Stunning picture quality and easy to set up.
4,Bluetooth Speaker,Compact and powerful with rich sound.


## LLMChain

In [23]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
#from langchain_core.runnables import RunnableLambda

In [24]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [25]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [26]:
#chain = LLMChain(llm=llm, prompt=prompt)
chain = llm  | prompt

In [27]:
product = "Wireless Headphones"
chain.invoke(product)

ChatPromptValue(messages=[HumanMessage(content="What is the best name to describe     a company that makes content='Wireless headphones are headphones that use Bluetooth technology to connect to devices like smartphones, tablets, or computers wirelessly. They allow users to listen to music or make phone calls without being tethered to their device by a cord. Wireless headphones come in various styles, including over-ear, on-ear, and in-ear options, and are available from a wide range of brands at different price points. They are popular for use during workouts, commutes, or anytime users want to enjoy audio without the hassle of cables.' response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 11, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f272a204-ac37-4591-9d38-03085e6d68e9-0' usage_metadata={'input_tokens': 11, 'output_tokens': 105, 'total_tokens': 116}?")])

## SimpleSequentialChain

In [28]:
from langchain.chains import SimpleSequentialChain

In [37]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
#chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_one = llm | first_prompt

chain_one.invoke(product)

ChatPromptValue(messages=[HumanMessage(content="What is the best name to describe     a company that makes content='Wireless headphones are a type of headphone that connects to a device, such as a smartphone or laptop, via Bluetooth or other wireless technology. This allows the user to listen to audio without being physically connected to the device with a cord. Wireless headphones are popular for their convenience and portability, as they allow for freedom of movement without being restricted by a cord. They are commonly used for activities such as exercising, commuting, or simply listening to music or podcasts on the go.' response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 11, 'total_tokens': 106}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-c747d3a9-a194-4473-8339-a3b2a896447a-0' usage_metadata={'input_tokens': 11, 'output_tokens': 95, 'total_tokens': 106}?")])

In [31]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
#chain_two = LLMChain(llm=llm, prompt=second_prompt)
chain_two = llm | second_prompt

In [51]:
#overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
#                                             verbose=True
#                                            )

from langchain_core.runnables import RunnableSequence  # , RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# output parser
output_parser = StrOutputParser()

#overall_simple_chain = chain_one | chain_two | output_parser

# combined chain
overall_simple_chain = RunnableSequence(
    {
        "company_name": chain_one,
    },
    second_prompt,
    llm,
    output_parser
)
print(overall_simple_chain)

result = overall_simple_chain.invoke(product)
print(result)

first={
  company_name: ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7c23dd2d07f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7c23dd2bce50>, temperature=0.9, openai_api_key=SecretStr('**********'), openai_proxy='')
                | ChatPromptTemplate(input_variables=['product'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], template='What is the best name to describe     a company that makes {product}?'))])
} middle=[ChatPromptTemplate(input_variables=['company_name'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name'], template='Write a 20 words description for the following     company:{company_name}'))]), ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7c23dd2d07f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7c23dd2bce50>, temperature=0.9, openai_api_key=SecretStr('******

## SequentialChain

In [52]:
from langchain.chains import SequentialChain

In [59]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to spanish:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= Spanish_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Spanish_Review"
                    )


In [60]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{Spanish_Review}"
)
# chain 2: input= Spanish_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )


In [67]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Spanish_Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [68]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [69]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["Spanish_Review", "summary","followup_message","language"],
    verbose=True
)

In [70]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Very useful features and long-lasting battery.',
 'Spanish_Review': 'Características muy útiles y batería de larga duración.',
 'summary': 'The review highlights very useful features and a long-lasting battery.',
 'followup_message': '¡Gracias por resaltar las funciones muy útiles y la batería de larga duración! Estamos encantados de que hayas encontrado útil esa información. Si necesitas más detalles sobre el producto, ¡no dudes en preguntar! ¡Estamos aquí para ayudarte!',
 'language': 'Spanish'}

## Router Chain

In [71]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""


englishtranslator_template = """You are a highly skilled English translator.\
You have a deep understanding of both source and target languages,\
a passion for languages and cultures, excellent communication skills,\
strong attention to detail, and the ability to convey the original tone and style.\
You are proficient in translating a wide range of texts accurately and effectively,\
ensuring that the meaning is preserved while making the text sound natural and fluent.\
You excel at handling idiomatic expressions, cultural nuances, and complex syntax.

Here is a text to translate:
{input}"""


In [72]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },

    {
    "name": "english translator",
    "description": "Good for translating texts accurately while preserving original meaning and style",
    "prompt_template": englishtranslator_template
}
]

In [73]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [74]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [75]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [76]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [77]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [78]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [79]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [80]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [81]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for the compliment! The answer to the question "what is 2 + 2" is 4.'

In [88]:
#chain.run("Why does every cell in our body contain DNA?")
chain.run("Traduza esta sentença para o Frances: Eu amo vocês minha familia linda")



> Entering new MultiPromptChain chain...
english translator: {'input': 'Translate this sentence to French: I love you my beautiful family'}
> Finished chain.


"Je t'aime ma belle famille."

Reminder: Download your notebook to you local computer to save your work.